In [135]:
import pandas as pd
import numpy as np

dados= np.arange(150,156,1)
dados

array([150, 151, 152, 153, 154, 155])

In [136]:
# numpy considera os intervalos acumulados e nao somas 
np.quantile(dados,[.25,.50,.75])

array([151.25, 152.5 , 153.75])

In [137]:
from scipy import stats

stats.scoreatpercentile(dados,50)

152.5

In [138]:
pd.DataFrame(dados).quantile([0.25,0.5])

,0
0.25,151.25
0.50,152.50



<h1 style="color:yellow;">Quartis para Dados Agrupados</h1>

In [139]:
dados = {'AlturasCM': np.repeat([150, 154, 158, 162, 166, 170], [5, 9, 11, 7, 5, 3])}

In [140]:
dataset = pd.DataFrame(dados)
dataset.head()

,AlturasCM
0,150
1,150
2,150
3,150
4,150


In [142]:
import sidetable

dataset_sum = dataset.stb.freq(['AlturasCM'])
dataset_sum


,AlturasCM,count,percent,cumulative_count,cumulative_percent
0,158,11,27.5,11,27.5
1,154,9,22.5,20,50.0
2,162,7,17.5,27,67.5
3,166,5,12.5,32,80.0
4,150,5,12.5,37,92.5
5,170,3,7.5,40,100.0


In [143]:
renam = {'count':'frequencia_absoluta','percent':'frequencia_relativa',
        'cumulative_count':'frequencia_acumulada','cumulative_percent':'frequencia_relativa_acumulado'}
dataset_sum = dataset_sum.rename(columns=renam)
dataset_sum['faixa_altura'] = pd.cut(dataset_sum['AlturasCM'],bins=8)
dataset_sum

,AlturasCM,frequencia_absoluta,frequencia_relativa,frequencia_acumulada,frequencia_relativa_acumulado,faixa_altura
0,158,11,27.5,11,27.5,"(157.5, 160.0]"
1,154,9,22.5,20,50.0,"(152.5, 155.0]"
2,162,7,17.5,27,67.5,"(160.0, 162.5]"
3,166,5,12.5,32,80.0,"(165.0, 167.5]"
4,150,5,12.5,37,92.5,"(149.98, 152.5]"
5,170,3,7.5,40,100.0,"(167.5, 170.0]"


In [144]:
# funcao que retorna o indice de cada classe
# atributo retorna o ponto medio 
dataset_sum['ponto'] = pd.IntervalIndex(dataset_sum['faixa_altura']).mid
dataset_sum

,AlturasCM,frequencia_absoluta,frequencia_relativa,frequencia_acumulada,frequencia_relativa_acumulado,faixa_altura,ponto
0,158,11,27.5,11,27.5,"(157.5, 160.0]",158.75
1,154,9,22.5,20,50.0,"(152.5, 155.0]",153.75
2,162,7,17.5,27,67.5,"(160.0, 162.5]",161.25
3,166,5,12.5,32,80.0,"(165.0, 167.5]",166.25
4,150,5,12.5,37,92.5,"(149.98, 152.5]",151.24
5,170,3,7.5,40,100.0,"(167.5, 170.0]",168.75


,AlturasCM,frequencia_absoluta,frequencia_relativa,frequencia_acumulada,frequencia_relativa_acumulado,faixa_altura,ponto
4,150,5,12.5,37,92.5,"(149.98, 152.5]",151.24
1,154,9,22.5,20,50.0,"(152.5, 155.0]",153.75
0,158,11,27.5,11,27.5,"(157.5, 160.0]",158.75
2,162,7,17.5,27,67.5,"(160.0, 162.5]",161.25
3,166,5,12.5,32,80.0,"(165.0, 167.5]",166.25
5,170,3,7.5,40,100.0,"(167.5, 170.0]",168.75



<b>25%</b> e representado por 1/4<br>
<b>50%</b> e representado por 2/4<br>
<b>75%</b> e representado por 3/4 <br>
4 sao total de cortes que fazemos no quartil 

q1 = (Limite inferior + soma frequancia absoluta/4  - Frequencia absoluta anterior / Soma Frquencia absoluta) * amplitude

In [145]:
total_freq_acum = dataset_sum['frequencia_acumulada'].tail(1)
Efi = total_freq_acum/4 # primeira parte da formula, soma frquencia acumlada/4 
Efi # sera a constante da formula, serve para acharmos onde esta o q1 

5    10.0
Name: frequencia_acumulada, dtype: float64

In [146]:
v = np.array(pd.IntervalIndex(dataset_sum['faixa_altura']).left)# limite inferiorb
v.sort()
v

array([149.98, 152.5 , 157.5 , 160.  , 165.  , 167.5 ])

In [255]:
# 1/4 representa 25 , 2/4 representa 50 , 3/4 75% 
# limite inferior e o menor limite do intervalo 
# preciso encontrar pela frequencia acumulada onde esta os 25% nesse caso no 11, pois 11/40 = 27% 
# os 25% esta contido nos 27 % 

'''
limite inferior (menor n)do intervalo + (soma Facumulada / 4 quartil(25%) ) - frequencia acumulada  anterior/
frequencia do intervalo* amplitude(164-168 = 4 )
'''
Q1 = (164+ 40/4-7 )/ 4 *4
Q1

167.0

In [256]:
q2 =  172 + (((40/2)-16)*4)/8
q2

174.0